In [13]:
import numpy as np
from scipy.optimize import minimize
import pandas as pd
from datetime import datetime

In [14]:
# Step 1: Define the risk scores for each protocol

# As an examples I will use 5 different Protocols
# 1 AAVE
# 2 UNISWAP
# 3 BALANCER
# 4 TRADER JOE
# 5 ORCA

# Define the protocol data
protocol_data = {
    'AAVE': {'TVL': 10.688, 'Start Date': '2020-05-20'},
    'UNISWAP': {'TVL': 5.569, 'Start Date': '2019-02-10'},
    'BALANCER': {'TVL': 1.043, 'Start Date': '2020-04-14'},
    'TRADER JOE': {'TVL': 0.184, 'Start Date': '2021-08-01'},
    'ORCA': {'TVL': 0.231, 'Start Date': '2021-04-16'}
}

# Calculate days of existence and ratings
ratings = []
for protocol, data in protocol_data.items():
    days = (datetime.now() - datetime.strptime(data['Start Date'], '%Y-%m-%d')).days
    rating = days * data['TVL']
    ratings.append(rating)

ratings_values = np.array(ratings)

inverted_risk_scores = 1 / ratings_values

In [15]:
# Step 2: Create the risk scores matrix (Σ)
Sigma = np.diag(inverted_risk_scores)

In [16]:
# Step 3: Normalize the risk scores matrix (Σ̃)
Sigma_norm = Sigma / np.linalg.norm(Sigma)

In [17]:
print(Sigma_norm)

[[0.00970424 0.         0.         0.         0.        ]
 [0.         0.01410433 0.         0.         0.        ]
 [0.         0.         0.09703533 0.         0.        ]
 [0.         0.         0.         0.80741701 0.        ]
 [0.         0.         0.         0.         0.58169477]]


In [18]:
# Step 4: Define the ERC weighting methodology
def risk_contribution(weights, Sigma):
    return np.dot(weights, np.dot(Sigma, weights))

In [23]:
def risk_contributions(weights, Sigma):
    total_risk = risk_contribution(weights, Sigma)
    marginal_contributions = np.dot(Sigma, weights)
    return weights * marginal_contributions / total_risk

In [24]:
def variance_risk_contributions(weights, Sigma):
    contributions = risk_contributions(weights, Sigma)
    return np.var(contributions)

In [25]:
# Step 5: Optimize the Weights

# Number of protocols
n = len(protocol_data)

# Initial guess: equal weights
initial_weights = np.ones(n) / n  # Initial guess: equal weights

# Constraints: Weights must sum to 1
constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})  # Weights must sum to 1

# Bounds: Weights must be between 0 and 1
bounds = [(0, 1) for _ in range(n)]  # Weights must be between 0 and 1

# Optimize to find the weights that minimize the variance of risk contributions
result = minimize(variance_risk_contributions, initial_weights, args=(Sigma_norm,),
                  method='SLSQP', bounds=bounds, constraints=constraints)

# Get the optimal weights
optimal_weights = result.x

In [26]:
# Step 6: Define the total allocation amount
total_allocation = 10000  # Total amount to be allocated in dollars

In [27]:
# Calculate the dollar allocation for each protocol
dollar_allocation = optimal_weights * total_allocation

In [28]:
# Step 7: Print the summary
protocols = ['AAVE', 'UNISWAP', 'BALANCER', 'TRADER JOE', 'ORCA']
print("Optimal Weights:", optimal_weights)
print("Variance of Risk Contributions:", variance_risk_contributions(optimal_weights, Sigma_norm))
print("\nSummary of Allocations:")
for protocol, weight, allocation in zip(protocols, optimal_weights, dollar_allocation):
    print(f"{protocol}: ${allocation:.2f} ({weight*100:.2f}%)")

Optimal Weights: [0.419956   0.34746797 0.13253519 0.04595866 0.05408218]
Variance of Risk Contributions: 1.644796412616186e-07

Summary of Allocations:
AAVE: $4199.56 (42.00%)
UNISWAP: $3474.68 (34.75%)
BALANCER: $1325.35 (13.25%)
TRADER JOE: $459.59 (4.60%)
ORCA: $540.82 (5.41%)
